In [21]:
import os
import argparse
import numpy as np
import pandas as pd
import time

import torch
from torch import optim
import torch.nn as nn

from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision.datasets as datasets
from torchvision import transforms
from data_loader import *
from model import *
from query import *

In [22]:
DEFAULT_DATASET_DIR = "./dataset/"
DEFAULT_MODEL_PATH = "./ckpt/model_ckpt.tar"

In [4]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)
print(len(siamese_dataset))

144


In [5]:
img0_file = siamese_dataset.genuine[0][1]
img1_file = siamese_dataset.genuine[1][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_50_1.png',
 './dataset/signatures/valid\\full_org\\original_50_10.png')

In [6]:
genuine_sigs = siamese_dataset.genuine
forged_sigs = siamese_dataset.forged

In [7]:
len(siamese_dataset)

144

In [8]:
query_model = QueryModel(DEFAULT_MODEL_PATH)

D:\Codes\axis-ai-challenge\model\model.py:72: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_in')


In [9]:
output = query_model.predict(img0, img1)

In [10]:
print(output.item())

0.6338170766830444


In [11]:
img0_file = siamese_dataset.genuine[63][1]
img1_file = siamese_dataset.forged[63][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_52_23.png',
 './dataset/signatures/valid\\full_forg\\forgeries_52_23.png')

In [12]:
output = query_model.predict(img0, img1)

In [13]:
print(output.item())

0.030160462483763695


In [14]:
img0_file = 'vin_real.jpg'
img1_file = 'vin_forged.jpg'
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('vin_real.jpg', 'vin_forged.jpg')

In [15]:
output = query_model.predict(img0, img1)

In [16]:
print(output.item())

0.7312096357345581


# CEDAR Dataset Scores

In [17]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)

In [18]:
len(siamese_dataset)

144

In [19]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
# for i in range(1, 10):
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.getConfidence(x1, x2)
    print(output.item())
#     log_probas = F.sigmoid(output)
#     print(output.item())
#     log_probas = F.sigmoid(output)
#     print(log_probas.item())

    if output >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif output <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif output >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif output <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()

tensor([ 0.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


0.017276505008339882
TRUE NEGATIVE

tensor([ 0.])

0.3423851728439331
TRUE NEGATIVE

tensor([ 1.])

0.23675696551799774
FALSE NEGATIVE

tensor([ 1.])

0.5612755417823792
TRUE POSITIVE

tensor([ 1.])

0.23587851226329803
FALSE NEGATIVE

tensor([ 1.])

0.5072474479675293
TRUE POSITIVE

tensor([ 1.])

0.641618013381958
TRUE POSITIVE

tensor([ 1.])

0.5020878314971924
TRUE POSITIVE

tensor([ 1.])

0.5561263561248779
TRUE POSITIVE

tensor([ 1.])

0.12104868143796921
FALSE NEGATIVE

tensor([ 0.])

0.05360248684883118
TRUE NEGATIVE

tensor([ 0.])

0.15320147573947906
TRUE NEGATIVE

tensor([ 1.])

0.5087356567382812
TRUE POSITIVE

tensor([ 1.])

0.5904037952423096
TRUE POSITIVE

tensor([ 1.])

0.5584030747413635
TRUE POSITIVE

tensor([ 1.])

0.40113261342048645
FALSE NEGATIVE

tensor([ 0.])

0.1831057071685791
TRUE NEGATIVE

tensor([ 1.])

0.5593693256378174
TRUE POSITIVE

tensor([ 0.])

0.09052753448486328
TRUE NEGATIVE

tensor([ 0.])

0.0764370709657669
TRUE NEGATIVE

tensor([ 0.])

0.132482

In [20]:
# tp = tp/10
# fp = fp/10
# tn = tn/10
# fn = fn/10
correct = tp+tn
print(100*correct/total)

print((tn/total)*100, (fp/total)*100)
print((fn/total)*100, (tp/total)*100)


83.33333333333333
54.166666666666664 0.6944444444444444
15.972222222222221 29.166666666666668


# Sample Dataset from Axis AI Scores

In [20]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'dataset3/train')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset, dataset_type='sample')

In [21]:
len(siamese_dataset)

140

In [22]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
# for i in range(0, 10):
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.model(x1, x2)
    print(output.item())
    log_probas = F.sigmoid(output)
    print(log_probas.item())

    if log_probas >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif log_probas <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif log_probas >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif log_probas <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()


tensor([ 1.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


-4.138515949249268
0.015696199610829353
FALSE NEGATIVE

tensor([ 1.])

-4.138515949249268
0.015696199610829353
FALSE NEGATIVE

tensor([ 1.])

0.03494611009955406
0.5087356567382812
TRUE POSITIVE

tensor([ 1.])

1.270080804824829
0.7807565927505493
TRUE POSITIVE

tensor([ 0.])

-9.36060619354248
8.604051254224032e-05
TRUE NEGATIVE

tensor([ 1.])

1.3094271421432495
0.7874172329902649
TRUE POSITIVE

tensor([ 1.])

0.7429184913635254
0.6776337027549744
TRUE POSITIVE

tensor([ 0.])

-6.60599946975708
0.001350404811091721
TRUE NEGATIVE

tensor([ 1.])

0.03494611009955406
0.5087356567382812
TRUE POSITIVE

tensor([ 1.])

0.4865950345993042
0.6193040013313293
TRUE POSITIVE

tensor([ 1.])

-1.795344591140747
0.14241869747638702
FALSE NEGATIVE

tensor([ 0.])

-16.98392677307129
4.207017667567925e-08
TRUE NEGATIVE

tensor([ 1.])

0.03494611009955406
0.5087356567382812
TRUE POSITIVE

tensor([ 0.])

-1.4943695068359375
0.1832667887210846
TRUE NEGATIVE

tensor([ 1.])

2.562056303024292
0.92837929725

In [23]:
# tp = tp/10
# fp = fp/10
# tn = tn/10
# fn = fn/10
correct = tp+tn
print(100*correct/total)

print(((tp)/total)*100, ((fp)/total)*100)
print(((tn)/total)*100, ((fn)/total)*100)


70.71428571428571
35.0 10.0
35.714285714285715 19.28571428571429
